In [172]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

URL = 'https://www.espn.com/wnba/playbyplay?gameId=401230763'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

In [211]:
times = [x.text for x in soup.find_all('td', class_='time-stamp')]
plays = [x.text for x in soup.find_all('td', class_='game-details')]
score = [x.text for x in soup.find_all('td', class_='combined-score')]
logourl = [x.find('img').get('src') for x in soup.find_all('td', class_='logo')]
team = [re.match(r"https.*/wnba/500/(?P<team>[a-z]{3})",x).group('team').upper() for x in logourl]

shotdescriptions = pd.DataFrame({'Time': times,
                      'EventDescription': plays,
                      'Score': score,
                      'Team': team})

In [212]:
shotdescriptions

,Time,EventDescription,Score,Team
0,10:00,Brittney Griner vs. Elizabeth Williams (Skylar...,0 - 0,PHX
1,9:40,Brianna Turner makes 7-foot two point shot (Sk...,2 - 0,PHX
2,9:26,Elizabeth Williams misses layup,2 - 0,ATL
3,9:26,Brianna Turner defensive rebound,2 - 0,PHX
4,9:10,Diana Taurasi misses 27-foot three point pullu...,2 - 0,PHX
...,...,...,...,...
396,25.2,Atlanta 20 Sec. timeout,81 - 72,ATL
397,22.4,Betnijah Laney makes layup (Chennedy Carter as...,81 - 74,ATL
398,11.3,Skylar Diggins-Smith vs. Glory Johnson,81 - 74,PHX
399,0.0,End of the 4th Quarter,81 - 74,ATL


In [175]:
awayshots = soup.find('ul', class_='shots away-team')
awayshots = awayshots.find_all('li')
awayshotchart = []
for row in awayshots:
    coordtext = row.get('style')
    match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
    xcoord = match.group('xcoord')
    ycoord = match.group('ycoord')
    ycoord = str(100-float(ycoord))
    row_data = [row.get('class')[0],
                row.get('data-homeaway'),
                row.get('data-period'),
                row.get('data-shooter'),
                row.get('data-text'),
                row.get('id'),
                xcoord,
                ycoord]
    awayshotchart.append(row_data)
    
homeshots = soup.find('ul', class_='shots home-team')
homeshots = homeshots.find_all('li')
homeshotchart = []
for row in homeshots:
    coordtext = row.get('style')
    match = re.match(r".*left:(?P<xcoord>[0-9\.]+)%;top:(?P<ycoord>[0-9\.]+)%;",coordtext)
    xcoord = match.group('xcoord')
    ycoord = match.group('ycoord')
    ycoord = str(100-float(ycoord))
    row_data = [row.get('class')[0],
                row.get('data-homeaway'),
                row.get('data-period'),
                row.get('data-shooter'),
                row.get('data-text'),
                row.get('id'),
                xcoord,
                ycoord]
    homeshotchart.append(row_data)

awayshotchart = pd.DataFrame(awayshotchart)
homeshotchart = pd.DataFrame(homeshotchart)

In [176]:
awayshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']
homeshotchart.columns = ['ShotOutcome','EventTeam','Quarter','pid','EventDescription','ShotID','xcoord','ycoord']

In [204]:
homeshotchart

,ShotOutcome,EventTeam,Quarter,pid,EventDescription,ShotID,xcoord,ycoord
0,missed,home,1,2566081,Elizabeth Williams misses layup,shot0,90.22222222222223,50.0
1,missed,home,1,2593770,Betnijah Laney misses 14-foot pullup jump shot,shot1,79.11111111111111,54.0
2,made,home,1,4280892,Chennedy Carter makes two point shot,shot2,91.33333333333333,50.0
3,missed,home,1,1110,Shekinna Stricklen misses 25-foot three point ...,shot3,69.11111111111111,74.0
4,missed,home,1,4280892,Brittney Griner blocks Chennedy Carter 's 5-fo...,shot4,90.22222222222223,58.0
...,...,...,...,...,...,...,...,...
57,missed,home,4,2593770,Betnijah Laney misses 22-foot pullup jump shot,shot66,69.11111111111111,48.0
58,made,home,4,4280892,Chennedy Carter makes 16-foot two point shot,shot69,86.88888888888889,80.0
59,missed,home,4,4280892,Chennedy Carter misses 29-foot three point jumper,shot70,64.66666666666667,72.0
60,made,home,4,2593770,Betnijah Laney makes 14-foot running pullup ju...,shot73,79.11111111111111,64.0


In [177]:
len(awayshotchart)

75

In [178]:
len(homeshotchart)

62

In [202]:
shotchartindex = []
for row in range(len(shotdescriptions)):
    misses = re.match('.*misses*',shotdescriptions.loc[row,'EventDescription'])
    makes = re.match('.*makes*',shotdescriptions.loc[row,'EventDescription'])
    if makes or misses:
        ft = re.match('.*free throw*',shotdescriptions.loc[row,'EventDescription'])
        if not ft:
            print(shotdescriptions.loc[row,'EventDescription'])
        shotchartindex.append(row)
        

Brianna Turner makes 7-foot two point shot (Skylar Diggins-Smith assists)
Elizabeth Williams misses layup
Diana Taurasi misses 27-foot three point pullup jump shot
Betnijah Laney misses 14-foot pullup jump shot
Skylar Diggins-Smith misses 28-foot three point pullup jump shot
Chennedy Carter makes two point shot
Skylar Diggins-Smith misses 8-foot two point shot
Shekinna Stricklen misses 25-foot three point pullup jump shot
Brittney Griner misses 12-foot jumper
Chennedy Carter makes layup
Brittney Griner makes 18-foot jumper (Sophie Cunningham assists)
Betnijah Laney makes 25-foot three pointer
Diana Taurasi misses 25-foot three point pullup jump shot
Elizabeth Williams misses 7-foot two point shot
Betnijah Laney misses 26-foot three point pullup jump shot
Skylar Diggins-Smith makes layup (Diana Taurasi assists)
Chennedy Carter makes 13-foot pullup jump shot
Sophie Cunningham misses 24-foot three point jumper
Bria Hartley misses 30-foot three point pullup jump shot
Alanna Smith makes 26-

In [203]:
len(shotchartindex)

168

In [200]:
shotdescriptions.loc[shotchartindex,'EventDescription']

1      Brianna Turner makes 7-foot two point shot (Sk...
2                        Elizabeth Williams misses layup
4      Diana Taurasi misses 27-foot three point pullu...
6         Betnijah Laney misses 14-foot pullup jump shot
8      Skylar Diggins-Smith misses 28-foot three poin...
                             ...                        
380    Chennedy Carter misses 29-foot three point jumper
387    Brittney Griner makes layup (Diana Taurasi ass...
388    Betnijah Laney makes 14-foot running pullup ju...
395    Brianna Turner makes layup (Diana Taurasi assi...
397    Betnijah Laney makes layup (Chennedy Carter as...
Name: EventDescription, Length: 133, dtype: object